In [26]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pandas as pd
import re

# Plotting tools
! pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

In [7]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus.
lda = gensim.models.LdaModel(common_corpus, num_topics=10)

In [90]:
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.drop_duplicates(inplace=True)
df = df.loc[df.Reviews.notna(), ] 
df = df.sample(n = 100000) 

In [91]:
df.shape

(100000, 6)

In [92]:
#Convert reviews to list
data = df.Reviews.values.tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<ipython-input-92-af4318837562>:4: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-92-af4318837562>:6: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [93]:
#Tokenize words and Clean-up Text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

In [87]:
len(data_words)

349710

In [94]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [65]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [66]:
# NLTK Stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
# spacy for lemmatization
import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
spacy.cli.download("en_core_web_md")
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

In [96]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops) #349,710 len

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) #took long long minutes

In [ ]:
data_lemmatized[:10]

In [98]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [107]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [108]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.291884  0.227377       1        1  29.797430
1     -0.088152 -0.193745       2        1  23.929003
4      0.203418 -0.203186       3        1  21.081623
2     -0.106446 -0.109433       4        1  17.790785
0      0.283065  0.278987       5        1   7.401159, topic_info=          Term           Freq          Total Category  logprob  loglift
60       phone  114996.000000  114996.000000  Default  30.0000  30.0000
36         use   25960.000000   25960.000000  Default  29.0000  29.0000
69        work   31011.000000   31011.000000  Default  28.0000  28.0000
5         good   29270.000000   29270.000000  Default  27.0000  27.0000
24       great   24761.000000   24761.000000  Default  26.0000  26.0000
..         ...            ...            ...      ...      ...      ...
849      nexus     883.780964     884.590424   Topic5  -4.9967   2.6026
274      piece     874.248794     875.054764   Topic5  -5.0076   2.6026
472  fantastic     864.867789     865.683874   Topic5  -5.0184   2.6026
233       high    1626.799122    1731.910276   Topic5  -4.3866   2.5409
351       type    1253.776900    1410.598168   Topic5  -4.6470   2.4857

[219 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
39        2  0.999786   able
155       5  0.999678    add
213       2  0.471984   also
213       3  0.024561   also
213       4  0.503331   also
...     ...       ...    ...
69        1  0.999988   work
70        1  0.104363  would
70        2  0.895149  would
70        3  0.000453  would
108       3  0.999630   year

[227 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 3, 1])